## 데이터 수집 & 전처리

In [28]:
#API 호출 및 데이터 품질 검사
import re
import os
import requests
import json
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
import faiss
from langchain_community.vectorstores import FAISS

In [7]:

# .env 파일 로드
load_dotenv()

# 환경 변수에서 API 키 불러오기
api_key = os.getenv("OPENAI_API_KEY")

# 파일 로드
file_path = "SourceCode/RAG/비상시국민행동요령(종합).PDF"
# PDF 파일 경로
loader = PyPDFLoader(file_path=file_path)

docs = loader.load()

In [8]:
def preprocess_text(text):
    '''전처리 함수
    설명 : 추가할 부분 추가하기'''
    # 1. 처음에 숫자가 나오면 제거
    text = re.sub(r'^\s*\d+\s*', '', text)    

    # 2. 불필요한 줄 바꿈과 공백 정리
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    return text

In [11]:
# 각 문서의 page_content 전처리
for doc in docs:
    doc.page_content = preprocess_text(doc.page_content)

In [12]:
recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

# 청크 확인 디버그 코드
for idx, chunk in enumerate(splits):
    print(f"Chunk {idx + 1}:")
    print("-" * 20)
    print(chunk.page_content)
    print("\n" + "=" * 40 + "\n")

Chunk 1:
--------------------
비상시 국민행동요령 CONTENTS 온 가족이 함께 안전하게 비상사태시 행동요령 화생방 피해대비 행동요령 인명·시설 피해시 행동요령 비상대비물자 준비 및 사용요령 •만화(웹툰)로 보는 02 비상시 국민행동요령 •비상시 행동요령 08 •민방공 경보 발령시 행동요령 11 •일상생활 비상대비 3가지 14 •화학무기 피해대비 행동요령 20 •생물학무기 피해대비 행동요령 21 •핵·방사능 피폭대비 행동요령 23 •핵·방사능 피폭대비 생존상식 25 •대형건물 붕괴·화재시 행동요령 32 •전기·물·가스 공급 중단시 행동요령 34 •지하철 피해시 행동요령 35 •인명·시설 피해복구 행동요령 37 •비상대비물자 준비요령 40 •화생방 대비물자 사용요령 42 •부상자 응급조치 요령 44 01 02 03 04 05


Chunk 2:
--------------------
비상시 국민행동요령 알아야 안전하다2 만화로 보는 비상시 국민 행동요령 온 가족이 함께 보고, 쉽고 재미있게 배워요! 가정에서, 학교에서, 집 밖에서 언제 어디서나 비상대비! 남녀노소 누구나 어렵지 않게 비상시 행동요령을 배워보아요. * 비상사태 정의 : 전시, 사변이나 이에 준하는 비상 시(비상대비에 관한 법률 제2조) 엄마가 불길한 꿈을 꾸었대 비상사태로 다치고, 우리 가족을 잃어버리는 꿈을 꾸었다더구나. 아빠, 엄마 뭐 하시는 거에요? 1 경계 경보가 울리면 이제 끝났다! 평상시 알고 대비하는게 가장 중요해. 당황하지 말고 침착하게 평소 준비해 놓았던 생활필수품을 확인해야 해요. 일상생활 중 잘 대비해 놓고 비상상황에선 침착하게 대응해야 해. 엄마가 공부한 비상시 행동요령들, 알려줄게!


Chunk 3:
--------------------
만화로 보는 비상시 국민행동요령 화생방 피해대비 행동요령 인명〮시설 피해시 행동요령 비상대비물자 준비 및 사용요령 비상사태시 행동요령 2 3 5 6 4 단전으로 조명이 꺼지면 단수에 대비하여 집안의 욕조, 양동이 등

## 임베딩

In [31]:
# 1. OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 2. 로컬 파일 저장소 설정 (사용자 환경에 맞는 경로로 설정)
store = LocalFileStore("F:/STUDY/sparta/999/박성규/emb")  # 로컬 경로 설정 각자 작성 해야합닏.

# 3. 캐시를 지원하는 임베딩 생성
cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=store,
    namespace=embeddings.model,  # 모델 이름을 네임스페이스로 설정
)

''' 디버깅 부분

# 4. 예시 텍스트
example_texts = [
    "OpenAI의 임베딩 모델은 강력한 자연어 처리를 제공합니다.",
    "로컬 캐시는 효율적인 데이터 처리를 가능하게 합니다.",
    "임베딩 모델을 활용하여 고품질의 애플리케이션을 개발할 수 있습니다."
]

# 5. 임베딩 생성 및 저장된 결과 출력
for text in example_texts:
    embedding = cached_embedder.embed_query(text)  # 임베딩 생성
    print(f"텍스트: {text}")
    print(f"임베딩 (첫 10개 값): {embedding[:10]}")  # 임베딩 값의 일부를 출력

print("임베딩 생성 완료 및 로컬 캐시에 저장되었습니다.")

'''

' 디버깅 부분\n\n# 4. 예시 텍스트\nexample_texts = [\n    "OpenAI의 임베딩 모델은 강력한 자연어 처리를 제공합니다.",\n    "로컬 캐시는 효율적인 데이터 처리를 가능하게 합니다.",\n    "임베딩 모델을 활용하여 고품질의 애플리케이션을 개발할 수 있습니다."\n]\n\n# 5. 임베딩 생성 및 저장된 결과 출력\nfor text in example_texts:\n    embedding = cached_embedder.embed_query(text)  # 임베딩 생성\n    print(f"텍스트: {text}")\n    print(f"임베딩 (첫 10개 값): {embedding[:10]}")  # 임베딩 값의 일부를 출력\n\nprint("임베딩 생성 완료 및 로컬 캐시에 저장되었습니다.")\n\n'

In [29]:
vectorstore = FAISS.from_documents(documents=splits, embedding=cached_embedder)

In [30]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4}) 